# Type Checking in Python

## Mike Driscoll

## Types in Python

- Python is strongly typed
- Python is dynamically typed

## Strongly Typed

Python is strongly typed in that you cannot merge different types:

```python
>>> "Mike" + 10
Traceback (most recent call last):
  Python Shell, prompt 1, line 1
builtins.TypeError: can only concatenate str (not "int") to str
```

## Dynamically Typed

Python allows variables to change types at runtime. Statically typed languages do not.

```python
>>> name = "Mike"
>>> name = 21
>>>
```

## Type Hinting History

- Python 3.5 - `typing` module released
- [PEP 484](https://www.python.org/dev/peps/pep-0484) and [PEP 483](https://www.python.org/dev/peps/pep-0483)
- Type hinting only. Python does not enforce

## Type Hinting Variables

In [ ]:
x: int  # a variable named x without initialization
y: float = 1.0  # a float variable, initialized to 1.0
z: bool = False
a: str = 'Hello type hinting'
d: dict[str, int] = {'one': 1}  # Python 3.9+ only

## Type Hinting Collections

Starting in 3.9, you can now use built-in collections: `list`, `dict`, `tuple`, and `set`

Prior to 3.9, you had to use this:

```python
from typing import List, Dict, Tuple, Set
```

## Type Hinting Functions / Methods

```python
def adder(x: int, y: int) -> None:
    print(f'The total of {x} + {y} = {x+y}')
```

## Variables with Multiple Types

Sometimes a variable can be of multiple types.

### Ye Olde Way

```python
from typing import Union

number: Union[int, float] = 10
```

### The Current Way

Starting in Python 3.10

```python
number: int | float = 10
```

## Type Hinting Optional

A variable can be one type or `None`

### Ye Olde Way

```python
from typing import Optional

number: Optional[int] = None
```

### The Current Way

Starting in Python 3.10

```python
number: int | None = 10
```

## More Complicated Type Hinting

- Callables
- Decorators
- Generators
- Complex Dictionaries

## Type Hinting Callables

Functions are first-class objects in Python, which means they can be used the same way as integers, dictionaries, etc.

In other words, you can pass a function or class around as well as return them!

In [3]:
from typing import Callable

def adder(a: int, b: int) -> int:
    return a + b

def caller() -> Callable[[int, int], int]:
    return adder

c = caller()
c(1, 2)

3

### Callable Parameters

Example: `Callable[[int, int], int]`

- `[int, int]` - The callable takes in two integer parameters
- `int` - The callable returns an integer

## Type Hinting Decorators

Most of our decorators use `# type: ignore` because they come from the tools repo

In [ ]:
# The wrong way
from functools import wraps
from typing import Any, Callable, TypeVar


Generic_function = TypeVar("Generic_function", bound=Callable[..., Any])

def info(func: Generic_function) -> Generic_function:
    @wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        print('Function name: ' + func.__name__)
        print('Function docstring: ' + str(func.__doc__))
        result = func(*args, **kwargs)
        return result
    return wrapper

@info
def doubler(number: int) -> int:
    """Doubles the number passed to it"""
    return number * 2

print(doubler(4))

In [ ]:
> mypy --strict .\info_decorator_paramspec.py
Success: no issues found in 1 source file

In [ ]:
# The right way (pre 3.12)
from functools import wraps
from typing import Callable, ParamSpec, TypeVar


P = ParamSpec("P")
R = TypeVar("R")

def info(func: Callable[P, R]) -> Callable[P, R]:
    @wraps(func)
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> R:
        print('Function name: ' + func.__name__)
        print('Function docstring: ' + str(func.__doc__))
        return func(*args, **kwargs)
    return wrapper

@info
def doubler(number: int) -> int:    
    return number * 2

print(doubler(4))

In [ ]:
# The right way 3.12+ (PEP 695)

from functools import wraps
from typing import Callable


def info[**P, R](func: Callable[P, R]) -> Callable[P, R]:
    @wraps(func)
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> R:
        print('Function name: ' + func.__name__)
        print('Function docstring: ' + str(func.__doc__))
        return func(*args, **kwargs)
    return wrapper

@info
def doubler(number: int) -> int:
    """Doubles the number passed to it"""
    return number * 2

print(doubler(4))

## Type Hinting Generators


```python
from typing import Generator

Generator[YieldType, SendType, ReturnType]
```

### Generator Type Hints

- `YieldType`: The type of values produced by `yield`.
- `SendType`: The type of values sent into the generator using the `.send()` method. If nothing is sent, use `None`.
- `ReturnType`: The type of the value returned when the generator finishes (caught as a `StopIteration` value). If no value is explicitly returned, use None. 

### Use Iterator for Simple Generators

If your generator only yields and does not send or returns values, you can use `Iterator`

In [1]:
from typing import Iterator

def infinite_stream(start: int) -> Iterator[int]:
    while True:
        yield start
        start += 1

## Type Hinting Complex Dictionaries

In [ ]:
from enum import IntEnum

class FordModel(IntEnum):
    F150 = 1
    Bronco = 2
    Mustang = 3

CAR_INFO = {
        FordModel.F150: {
            "name": "F-150",
            "retail": 38.6,
            'inventory': "trucks_2025.log",
        },
        FordModel.Bronco: {
            "name": "Bronco",
            "retail": 40.4,
            'inventory': "suvs_2025.log",
        },
        FordModel.Mustang: {
            "name": "Mustang",
            "retail": 32.5,
            'inventory': "cars_2025.log",
        }
}

In [ ]:
from enum import IntEnum

class FordModel(IntEnum):
    F150 = 1
    Bronco = 2
    Mustang = 3

CAR_INFO = {
        FordModel.F150: {
            "name": "F-150",
            "retail": 38.6,
            'inventory': "trucks_2025.log",
        },
        FordModel.Bronco: {
            "name": "Bronco",
            "retail": 40.4,
            'inventory': "suvs_2025.log",
        },
        FordModel.Mustang: {
            "name": "Mustang",
            "retail": 32.5,
            'inventory': "cars_2025.log",
        }
}
# Adding this causes type checkers to fail
price_increase = (CAR_INFO[FordModel.Mustang]["retail"]+10)

## Type checking the dict

```python
> ty check .\type_hinting_complex_dicts.py
error[unsupported-operator]: Unsupported `+` operation
  --> type_hinting_complex_dicts.py:27:7
   |
26 | print(CAR_INFO[FordModel.Mustang]["inventory"])
27 | print(CAR_INFO[FordModel.Mustang]["retail"]+10)
   |       -------------------------------------^--
   |       |                                     |
   |       |                                     Has type `Literal[10]`
   |       Has type `Unknown | str | int | float`
   |
info: rule `unsupported-operator` is enabled by default

Found 1 diagnostic>
```

### Use a TypedDict

In [ ]:
from enum import IntEnum
from typing import TypedDict

class FordModel(IntEnum):
    F150 = 1
    Bronco = 2
    Mustang = 3

class FordModelType(TypedDict):
    name: str
    retail: float
    inventory: str

In [ ]:
f150: FordModelType = {
            "name": "F-150",
            "retail": 38.6,
            'inventory': "trucks_2025.log",
        }

bronco: FordModelType =  {
            "name": "Bronco",
            "retail": 40.4,
            'inventory': "suvs_2025.log",
        }

mustang: FordModelType =  {
            "name": "Mustang",
            "retail": 32.5,
            'inventory': "cars_2025.log",
        }

CAR_INFO: dict[FordModel, FordModelType] = {
        FordModel.F150: f150,
        FordModel.Bronco: bronco,
        FordModel.Mustang: mustang
}

print(CAR_INFO[FordModel.Mustang]["inventory"])
print(CAR_INFO[FordModel.Mustang]["retail"]+10)

## Creating New Types

Create distinct new types to prevent logical errors

```python
from typing import NewType
```

In [1]:
from typing import NewType

UserId = NewType("UserId", int)

def get_user_name(user_id: UserId) -> str:
    return f"User-{user_id}"

# Fails the type checker
user_b = get_user_name(123)

## Hinting Vs Checking

Python type hinting is just that: Hinting! Python does not enforce types at all!

## Python Type Checkers

- Dropbox's [mypy](https://mypy.readthedocs.io/en/stable/index.html)
- Microsoft's [pyright](https://github.com/microsoft/pyright)
- [pyrefly](https://pyrefly.org/)
- [Astral's ty](https://docs.astral.sh/ty/)

## How do Python Type Checkers Work?

Static analysis and

- Before 3.10, you could use `__annotations__`
- 3.10+ - Use `inspect.get_annotations()`
- 3.14+ - Use `annotationlib.get_annotations()`

## Configuring Type Checkers

- pyproject.toml (`tool.mypy` or `tool.
- mypy's specific config - **mypy.ini**
- ty specific config - **ty.toml**

## mypy Detects Untyped Code

- `mypy --strict`
- `mypy --disallow-untyped-calls --disallow-untyped-defs --disallow-incomplete-defs --check-untyped-defs`


## ty and Untyped Code

Astral's ty does **not** currently detect untyped code. However, Ruff can detect untyped code if you enable [flake8-annotations](https://docs.astral.sh/ruff/rules/#flake8-annotations-ann)

## ?? Questions ??

References:

- https://stackoverflow.com/questions/11328920/is-python-strongly-typed
- https://peps.python.org/pep-0695/ - Weird new square brackets in function defs (AKA variance inference)
- https://peps.python.org/pep-0612/ - ParamSpec and Concatenate
- Python spec for type system - https://typing.python.org/en/latest/spec/